# Лабораторная работа №3
## Выполнил студент группы БФИ2001 Семёнов Андрей Владиславович

### Оглавление
1. [Задание 1](#Задание-№1)
2. [Задание 2](#Задание-№2)
4. [Вывод](#Вывод)

### Задание №1
Реализовать методы поиска подстроки в строке. Добавить возможность ввода строки и подстроки с клавиатуры. Предусмотреть возможность существования пробела. Реализовать возможность выбора опции чувствительности или нечувствительности к регистру. Оценить время работы каждого алгоритма поиска и сравнить его со временем работы стандартной функции поиска, используемой в выбранном языке программирования.

In [1]:
from __future__ import annotations
from collections import defaultdict
from typing import Callable
from time import time

#### Алгоритм Кнута-Морриса-Пратта

In [2]:
def kmp_search(source: str, template: str) -> int:
    prefix_list: list[int] = prefix(template)
    k: int = 0

    for i in range(len(source)):
        while k > 0 and template[k] != source[i]:
            k = prefix_list[k-1]
        if template[k] == source[i]:
            k += 1
        if k == len(template):
            return i - len(template) + 1

    return -1

In [3]:
def prefix(s: str) -> list[int]:
    v: list[int] = [0] * len(s)

    for i in range(1, len(s)):
        k: int = v[i-1]

        while k > 0 and s[k] != s[i]:
            k = v[k-1]

        if s[k] == s[i]:
            k = k + 1

        v[i] = k

    return v

#### Упрощенный алгоритм Бойера-Мура

In [4]:
def bm_search(source: str, template: str) -> int:
    i: int = len(template) - 1
    j: int = i
    k: int = i

    offset_dict: defaultdict[str, int] = get_offset_dict(template)

    while j >= 0 and i <= len(source) - 1:
        j = len(template) - 1
        k = i

        while j >= 0 and source[k] == template[j]:
            k -= 1
            j -= 1

        i += offset_dict[source[i]]

    if k < len(source) - len(template):
        return k + 1
    else:
        return -1

In [5]:
def get_offset_dict(template: str) -> defaultdict[str, int]:
    offset_dict: defaultdict[str, int] = defaultdict(lambda: len(template))

    for i, char in enumerate(template):
        offset_dict[char] = len(template) - i - 1;

    return offset_dict

In [6]:
def default_search(source: str, template: str) -> int:
    return source.index(template)

### Client

In [7]:
def client(searches_dict: dict[int, Callable]) -> None:
    print("Choose search method and input it's number")
    print("1. Knuth-morris-pratt algorithm")
    print("2. Boer-Moore algorithm")
    print("3. Default python algorithm")
    search_number: int = int(input())

    print("Case sensitive search? (yes/no)")
    case_sensetive: str = input()

    src: str = input("Input source string: ")
    target: str = input("Target substring: ")

    if _process_case_sense(case_sensetive):
        print(f"Result: {searches_dict[search_number](src, target)}")
    else:
        print(f"Result: {searches_dict[search_number](src.lower(), target.lower())}")

In [8]:
def _process_case_sense(case_sense: str) -> bool:
    return case_sense == "yes"

### Tests

In [9]:
search_methods: dict[int, Callable] = dict({
    1: kmp_search,
    2: bm_search,
    3: default_search
})

In [10]:
tests: tuple[tuple[str, str], ...] = tuple([
    (" abcdabcabcdabcdab", " abcd"),
    ("ycknFLkLBX", "kLBX"),
    ("LmILOrSWuy", "LmILOrSWuy"),
    ("eTqhWalKeP", "WalKeP"),
    ("HjPBLQxvot", "HjPBLQxv"),
    ("JvXXcuYBau", "YBau"),
    ("HTwaUWfNSR", "HTwaUWfNS"),
    ("BVsLtQmxFU", "VsLtQmxFU"),
    ("euOgxoSqaK", "gxo"),
    ("JNMgIgXxbd", "JNM"),
    ("heTnDoqwVj", "DoqwVj"),
])

In [11]:
time_results: defaultdict[int, set[float]] = defaultdict(lambda: set())

In [12]:
for key, method in search_methods.items():
    print(f"Start testing {method.__name__}")
    for before in tests:
        for _ in range(1000):
            start_time: float = time()
            after: int = method(*before)
            time_results[key].add(time() - start_time)
            mustbe: int = before[0].index(before[1])
            assert after == mustbe, f"{before=}, {after=}, {mustbe=}, {method}"
    print(f"Tests passed\n")

Start testing kmp_search
Tests passed

Start testing bm_search
Tests passed

Start testing default_search
Tests passed



In [13]:
for key, value in time_results.items():
    print(
        f"Average time for {search_methods[key].__name__}: "
        f"{sum(value) / len(tests)} ns"
    )

Average time for kmp_search: 0.0002137314189564098 ns
Average time for bm_search: 0.00027342276139692825 ns
Average time for default_search: 2.4123625321821734e-05 ns


In [14]:
client(search_methods)

Choose search method and input it's number
1. Knuth-morris-pratt algorithm
2. Boer-Moore algorithm
3. Default python algorithm
1
Case sensitive search? (yes/no)
no
Input source string: aBcabcabc
Target substring: abc
Result: 0


### Задание №2
Написать программу, определяющую, является ли данное
расположение «решаемым», то есть можно ли из него за конечное число
шагов перейти к правильному. Если это возможно, то необходимо найти хотя
бы одно решение - последовательность движений, после которой числа будут
расположены в правильном порядке.
#### Входные данные: массив чисел, представляющий собой расстановку в
Порядке «слева направо, сверху вниз». Число 0 обозначает пустое поле.
Например, массив [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0] представляет
собой «решенную» позицию элементов.
#### Выходные данные: если решения нет, то функция должна вернуть
Пустой массив []. Если решение есть, то необходимо представить решение —
для каждого шага записывается номер передвигаемого на данном шаге
элемента. 

In [15]:
import functools
from heapq import heappop, heappush
from math import sqrt
from typing import Callable

In [16]:
class Board:
    def __init__(self, board: list[int], history: list[Board] = None):
        if history is None:
            history = []

        self.board: list[int] = board
        self.side_len: int = int(sqrt(len(board)))
        self.history: list[Board] = history


    def get_neighbours(self) -> list[Board]:
        zero_index = self.board.index(0)

        neighs: list[list[int]] = [
            processor(zero_index)
            for processor in self._get_neighs_processors()
        ]

        self.history.append(self)
        return [Board(state, self.history) for state in neighs if state]


    def _get_neighs_processors(self) -> tuple[Callable]:
        return tuple([
            self._left_neighs,
            self._right_neighs,
            self._bot_neighs,
            self._top_neighs
        ])

    def _left_neighs(self, zero_index: int) -> list[int]:
        if zero_index > 0:
            if self._heuristic(zero_index-1, zero_index) == 1:
                return self._calculate_new_state(zero_index, zero_index-1)

        return []

    def _right_neighs(self, zero_index: int) -> list[int]:
        if zero_index < len(self.board)-1:
            if self._heuristic(zero_index+1, zero_index) == 1:
                return self._calculate_new_state(zero_index+1, zero_index)

        return []

    def _bot_neighs(self, zero_index: int) -> list[int]:
        if zero_index+self.side_len < len(self.board):
            if self._heuristic(zero_index+self.side_len, zero_index) == 1:
                return self._calculate_new_state(
                    zero_index+self.side_len, zero_index
                )

        return []

    def _top_neighs(self, zero_index: int) -> list[int]:
        if zero_index-self.side_len+1 > 0:
            if self._heuristic(zero_index-self.side_len, zero_index) == 1:
                return self._calculate_new_state(
                    zero_index-self.side_len, zero_index
                )

        return []

    def _calculate_new_state(self, i: int, j: int) -> list[int]:
        new_state = self.board.copy()
        new_state[i], new_state[j] = new_state[j], new_state[i]
        return new_state

    def __lt__(self, board: Board) -> bool:
        return self._board_manhattan() < board._board_manhattan()

    @functools.lru_cache
    def _board_manhattan(self) -> int:
        res: int = 0

        for current in range(len(self.board)):
            target: int = (self.board[current]-1) % len(self.board)
            res += self._heuristic(current, target)

        return res

    def _heuristic(self, current: int, target: int) -> int:
        s: int = self.side_len
        return abs(target % s - current % s) + abs(target // s - current // s)

In [17]:
def solution(board: Board):
    node_hash: dict[tuple[int, ...], int] = {}
    chains_queue: list[Board] = []
    target: tuple[int] = tuple([*[i for i in range(1, 16)], 0])

    heappush(chains_queue, board)

    while chains_queue:
        cur_chain = heappop(chains_queue)
        cur_node = tuple(cur_chain.board)

        if cur_node == target:
            return cur_chain

        node_hash[cur_node] = len(cur_chain.history)

        for chain in cur_chain.get_neighbours():
            key: tuple[int] = tuple(chain.board)
            if key in node_hash:
                if len(chain.history) >= node_hash[key]:
                    continue
                node_hash[key] = len(chain.history)

            heappush(chains_queue, chain)

    raise Exception(f"No solution for {board=}")

In [18]:
board: list[int] = [5, 1, 9, 3, 11, 13, 6, 8, 14, 10, 4, 15, 0, 12, 7, 2]
start = Board(board)

In [19]:
print("Start solving...")
result = solution(start)
print("Solved")

Start solving...
Solved


In [20]:
print(f"Steps count: {len(result.history)}\n")
for node in result.history:
    print(node.board)
print(result.board)

Steps count: 1453

[5, 1, 9, 3, 11, 13, 6, 8, 14, 10, 4, 15, 0, 12, 7, 2]
[5, 1, 9, 3, 11, 13, 6, 8, 14, 10, 4, 15, 12, 0, 7, 2]
[5, 1, 9, 3, 11, 13, 6, 8, 0, 10, 4, 15, 14, 12, 7, 2]
[5, 1, 9, 3, 11, 13, 6, 8, 14, 10, 4, 15, 12, 7, 0, 2]
[5, 1, 9, 3, 11, 13, 6, 8, 14, 10, 4, 15, 12, 7, 2, 0]
[5, 1, 9, 3, 11, 13, 6, 8, 14, 10, 4, 0, 12, 7, 2, 15]
[5, 1, 9, 3, 11, 13, 6, 8, 14, 10, 0, 4, 12, 7, 2, 15]
[5, 1, 9, 3, 11, 13, 6, 8, 14, 10, 2, 4, 12, 7, 0, 15]
[5, 1, 9, 3, 11, 13, 6, 8, 14, 10, 2, 4, 12, 7, 15, 0]
[5, 1, 9, 3, 11, 13, 6, 8, 14, 10, 2, 4, 12, 0, 7, 15]
[5, 1, 9, 3, 11, 13, 6, 8, 14, 10, 2, 0, 12, 7, 15, 4]
[5, 1, 9, 3, 11, 13, 6, 8, 14, 10, 2, 4, 0, 12, 7, 15]
[5, 1, 9, 3, 11, 13, 6, 8, 0, 10, 2, 4, 14, 12, 7, 15]
[5, 1, 9, 3, 11, 13, 6, 8, 10, 0, 2, 4, 14, 12, 7, 15]
[5, 1, 9, 3, 11, 13, 6, 8, 10, 2, 0, 4, 14, 12, 7, 15]
[5, 1, 9, 3, 11, 13, 6, 8, 10, 2, 7, 4, 14, 12, 0, 15]
[5, 1, 9, 3, 11, 13, 6, 8, 10, 2, 7, 4, 14, 12, 15, 0]
[5, 1, 9, 3, 11, 13, 6, 8, 10, 2, 7, 4, 14, 0,

[1, 2, 3, 8, 5, 6, 7, 4, 13, 11, 15, 0, 14, 10, 9, 12]
[1, 2, 3, 8, 5, 6, 7, 4, 13, 10, 11, 15, 0, 14, 9, 12]
[1, 2, 3, 8, 5, 6, 7, 4, 10, 14, 0, 15, 13, 9, 11, 12]
[1, 2, 3, 8, 5, 6, 7, 4, 10, 14, 15, 0, 13, 9, 11, 12]
[1, 2, 3, 8, 5, 6, 7, 4, 10, 14, 15, 12, 13, 9, 11, 0]
[1, 2, 3, 8, 5, 6, 7, 4, 10, 14, 15, 12, 13, 9, 0, 11]
[1, 2, 3, 8, 5, 6, 7, 4, 10, 14, 0, 12, 13, 9, 15, 11]
[1, 2, 3, 8, 5, 6, 7, 4, 10, 14, 12, 0, 13, 9, 15, 11]
[1, 2, 3, 8, 5, 6, 7, 4, 10, 14, 12, 11, 13, 9, 15, 0]
[1, 2, 3, 8, 5, 6, 7, 4, 10, 14, 12, 11, 13, 9, 0, 15]
[1, 2, 3, 8, 5, 6, 7, 4, 10, 14, 0, 11, 13, 9, 12, 15]
[1, 2, 3, 8, 5, 6, 7, 0, 10, 14, 11, 4, 13, 9, 12, 15]
[1, 2, 3, 8, 5, 6, 7, 4, 13, 10, 12, 9, 14, 0, 15, 11]
[1, 2, 3, 8, 5, 6, 7, 4, 13, 10, 11, 15, 14, 0, 9, 12]
[1, 2, 3, 8, 5, 6, 7, 4, 13, 10, 11, 15, 14, 9, 0, 12]
[1, 2, 3, 8, 5, 6, 7, 4, 13, 10, 11, 15, 14, 9, 12, 0]
[1, 2, 3, 8, 5, 6, 7, 4, 13, 10, 11, 0, 14, 9, 12, 15]
[1, 2, 3, 8, 5, 6, 7, 0, 10, 14, 15, 4, 13, 9, 11, 12]
[1, 2, 3, 

### Вывод

В данной лабораторной работе были изучены методы поиска строк, работающие за линейное время. Для проверки были реализованы тесты базовых случаев с замером среднего времени выполнения. Кроме этого был реализован CLI клиент, позволяющий вручную проверить работу каждого поиска. 

По итогу наилучшей скоростью обладает встроенная в python функция поиска, на втором месте находится упрощённый алгоритм Бойера-Мура, а самым медленным оказался алгоритм Кнута-Морриса-Пратта.

Кроме алгоритмов поиска было реализовано решение задачи про игры в пятнашки, в котором использовался алгоритм жадного поиска в ширину. Данный алгоритм был выбран по причине необходимости найти любое решение, а не оптимальное. При этом для ускорения поиска используется эвристическая функция, вычисляющая Манхэтеннское расстояние.